## Supertrees do not satisfy (J1)

In [1]:
using HomotopyContinuation
using DynamicPolynomials
@polyvar z w x[1:2] y[1:2] a[1:2] b[1:2] λᴿ λᴵ t s ν ν₁ ν₂
H = z^5*w^2 + 2*z^2*w - 2*z^3*w + 4*w + z - 2

z⁵w² - 2z³w + 2z²w + z + 4w - 2

In [2]:
# H: denominator
function critical_points(H)
    sys = System([H; w*differentiate(H, w) - z*differentiate(H, z)])
    sols = solutions(solve(sys))
    certs = certificates(certify(sys, sols))
    display(map(sol -> round.(sol; digits=3), solution_approximation.(certs)))
    return certs
end
C = critical_points(H)
println()

Tracking 7 paths... 100%|███████████████████████████████| Time: 0:00:07
  # paths tracked:                  7
  # non-singular solutions (real):  2 (2)
  # singular endpoints (real):      3 (3)
  # total solutions (real):         5 (5)


2-element Vector{Vector{ComplexF64}}:
 [-1.236 + 0.0im, 0.327 + 0.0im]
 [3.236 + 0.0im, 0.048 + 0.0im]

In [3]:
# split H(vars) = u(a, b) + iv(a, b)
function split(H, vars)
    @polyvar I
    u = sum(filter(t -> iseven(degree(t, I)), terms(H(vars => a + I.*b))))
    v = sum(filter(t -> isodd(degree(t, I)), terms(H(vars => a + I.*b))))
    return u, v = subs(u, I => im), -im*subs(v, I => im)
end

# H: demoninator, C: critical points
function non_comb_system(H, C)
    u, v = split(H, [z, w])
    criteqs = [ u; v; a.*differentiate(u, a) + b.*differentiate(u, b) .- λᴿ;
                      a.*differentiate(v, a) + b.*differentiate(v, b) .- λᴵ ]
    circeqs = [u([a; b] => [x; y]); v([a; b] => [x; y]); x.^2 + y.^2 - t.*(a.^2 + b.^2)]
    J2eqs = (ν₁.*y - ν₂.*x) .* differentiate(u([a; b] => [x; y]), x) - 
            (ν₁.*x + ν₂.*y) .* differentiate(u([a; b] => [x; y]), y)
    extended_systems = [
        System([criteqs; circeqs; s*(1-t) - 1; subs(J2eqs, [ν₁, ν₂] => [1, ν])],
            variables=[a; b; t; λᴿ; λᴵ; x; y; s; ν]),
        System([criteqs; circeqs; s*(1-t) - 1; subs(J2eqs[1], [ν₁, ν₂] => [0, 1])],
            variables=[a; b; t; λᴿ; λᴵ; x; y; s])
    ]
    
    for j in 1:2
        sols = solutions(solve(extended_systems[j]; show_progress=true))
        certs = certificates(certify(extended_systems[j], sols))
        certified_real = filter(is_real, certs)
        display(map(sol -> round.(sol[1:7]; digits=4), solution_approximation.(certified_real)))
        println()
    end
end
certs = non_comb_system(H, C)
println()

Tracking 57122 paths... 100%|███████████████████████████| Time: 0:21:42
  # paths tracked:                  57122
  # non-singular solutions (real):  280 (6)
  # singular endpoints (real):      1469 (36)
  # total solutions (real):         1749 (42)


6-element Vector{Vector{ComplexF64}}:
 [-1.2361 + 0.0im, 0.3273 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 2.4894 + 0.0im, 2.9271 + 0.0im, 0.0 + 0.0im]
 [-1.2361 + 0.0im, 0.3273 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 2.4894 + 0.0im, 2.9271 + 0.0im, 0.0 + 0.0im]
 [-1.2361 + 0.0im, 0.3273 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.6245 + 0.0im, 2.9271 + 0.0im, 0.0 + 0.0im]
 [-1.2361 + 0.0im, 0.3273 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.6245 + 0.0im, 2.9271 + 0.0im, 0.0 + 0.0im]
 [-1.2361 + 0.0im, 0.3273 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 2.2819 + 0.0im, 2.9271 + 0.0im, 0.0 + 0.0im]
 [-1.2361 + 0.0im, 0.3273 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 2.2819 + 0.0im, 2.9271 + 0.0im, 0.0 + 0.0im]

In [4]:
# (J1) is not satisfied.
sys = [H; w*differentiate(H, w) - z*differentiate(H, z)]
subs(differentiate(sys, [z, w]), [z, w] => [2, 1/8])

2×2 Matrix{Polynomial{true, Float64}}:
 0.25  4.0
 0.25  4.0